# Implementation of NCSN (Noise Conditional Score Networks)

In [1]:
%load_ext autoreload
%autoreload 2
!nvidia-smi
!which python

Mon Oct 21 19:31:27 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.86.10              Driver Version: 535.86.10    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-32GB           On  | 00000004:04:00.0 Off |                    0 |
| N/A   47C    P0              41W / 300W |     19MiB / 32768MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
try:
    from NCSN.train import train, make_dataset, cal_noise_level, langevin
    from NCSN.utils import train_set, val_set
    from NCSN.model import UNet
except:
    from train import train, make_dataset, cal_noise_level, langevin
    from utils import train_set, val_set
    from model import UNet

from torch.utils.data import DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F
import datetime
import os

train_loader = DataLoader(train_set, 256, shuffle=True, drop_last=False, pin_memory=True)
val_loader = DataLoader(val_set, 500, shuffle=True, drop_last=False, pin_memory=True)

def timestr():
    now = datetime.datetime.now()
    return now.strftime("%Y%m%d_%H%M%S")

Files already downloaded and verified
Files already downloaded and verified


In [3]:
# Here are the hyperparameters

epochs = 100
criterion = nn.MSELoss()
init_sigma = 28
final_sigma = 0.01
n_sigma = 75
eps = 1e-5
T = 5
eval_freq = "square"

model = UNet(L=n_sigma)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

print("model:", model)
# calculate the number of parameters
num_params = sum(p.numel() for p in model.parameters())
print("number of parameters:", num_params)

sigmas = cal_noise_level(init_sigma, final_sigma, n_sigma)

time_str = timestr()
print("time string:", time_str)

train(epochs=epochs, model=model, optimizer=optimizer, criterion=criterion, train_loader=train_loader, val_loader=val_loader, sigmas=sigmas, eps=eps, T=T, time_str=time_str, eval_freq=eval_freq)

model: UNet(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_up1): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_up2): Conv2d(32, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_up3): Conv2d(8, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (up1): ConvTranspose2d(256, 64, kernel_size=(2, 2), stride=(2, 2))
  (up2): ConvTranspose2d(32, 16, kernel_size=(2, 2), stride=(2, 2))
  (CIN1): ConditionalInstanceNorm2dPlus(
    (instance_norm): InstanceNorm2d(1, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (embed): Embedding(75, 3)
  )
  (CIN2): ConditionalInstanceNorm2dPlus(
    (instance_norm): InstanceNorm2d(16, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (embed): Embedding(75, 48)
  )
  (CIN3): Condition

Epoch 0: train loss 0.8689806182333764


Corruption MSE: 0.063820, Recovered MSE: 3.458826: 100%|███████████████████████████| 2000/2000 [00:15<00:00, 125.01it/s]
Epoch 2/100:   1%|▍                                                    | 2/235 [00:00<00:21, 10.74it/s, Train Loss=1.29]

Current best MSE: inf -> 3.458825744628906


Epoch 1: train loss 0.6226312480074294


...

Epoch 99/100:   0%|                                                          | 0/235 [00:00<?, ?it/s, Train Loss=0.0846]

Epoch 97: train loss 0.08321346529620759


Epoch 100/100:   0%|                                                         | 0/235 [00:00<?, ?it/s, Train Loss=0.0753]

Epoch 98: train loss 0.08185544445159587


Epoch 99: train loss 0.0820545716171569


In [4]:
# # load a model
# import os
# model = UNet(L=n_sigma).cuda()
# print(sum(p.numel() for p in model.parameters() if p.requires_grad))
# eval_epoch = 70
# model.load_state_dict(torch.load(f'NCSN/models/{eval_epoch}.pth'))
# make_dataset(model, sigmas, eps=eps, T=T)
# print("Dataset created")
# os.system("python NCSN/evaluate.py")

In [5]:
# sampling
try:
    from NCSN.utils import val_set
    from NCSN.model import UNet
    from NCSN.train import evaluate_denoising, cal_noise_level, langevin

except:
    from utils import val_set
    from model import UNet
    from train import evaluate_denoising, cal_noise_level, langevin

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

# val_loader = DataLoader(val_set, 500, shuffle=True, drop_last=False, pin_memory=True)

criterion = nn.MSELoss()
init_sigma = 1
final_sigma = 0.01
n_sigma = 10
eps = 2e-5
T = 100

sigmas = cal_noise_level(init_sigma, final_sigma, n_sigma)

model = UNet(L=n_sigma).cuda()
time_str = "20241009_173109"
eval_epoch = 5
model.load_state_dict(torch.load(f"/nobackup/users/sqa24/NCSN/{time_str}/models/{eval_epoch:03d}.pth"))
model.to('cuda')

# x = torch.rand(10, 1, 28, 28).cuda()
x = torch.randn(10, 1, 28, 28).cuda()
x = (x+1)/2
y = langevin(model, x, sigmas, eps=eps, T=T, save=True, epochs=eval_epoch, clamp=False, time_str=time_str)

In [4]:
# experiment on recovering
try:
    from NCSN.utils import val_set
    from NCSN.model import UNet
    from NCSN.train import evaluate_denoising, cal_noise_level  

except:
    from utils import val_set
    from model import UNet
    from train import evaluate_denoising, cal_noise_level

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

val_loader = DataLoader(val_set, 500, shuffle=True, drop_last=False, pin_memory=True)

criterion = nn.MSELoss()
init_sigma = 28
final_sigma = 0.01
n_sigma = 75
eps = 6e-6
T = 10

sigmas = cal_noise_level(init_sigma, final_sigma, n_sigma)

model = UNet(L=n_sigma).cuda()
time_str = "20241009_193254"
eval_epoch = 10
model.load_state_dict(torch.load(f"/nobackup/users/sqa24/NCSN/{time_str}/models/{eval_epoch:03d}.pth"))
model.to('cuda')

corruption_mse, mse, original, broken, recovered = evaluate_denoising(
    model, sigmas, eps=eps, T=T, val_loader=val_loader, outdir=f'NCSN/rubbish.out'
)

Corruption MSE: 0.063588, Recovered MSE: 26.811432: 100%|███████████████████████████| 2000/2000 [00:31<00:00, 62.95it/s]
